In [57]:
import os
import json
import requests
import re
import numpy as np
import faiss
import pickle
import gzip
from bs4 import BeautifulSoup
from tqdm import tqdm
import openai
from openai.embeddings_utils import get_embedding, get_embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open("openai_key.txt", "r") as f:
    openai.api_key = f.read().strip()

In [ ]:
# 爬取所有2866个连接
urls0 = [f"https://gwins.org/cn/milesguo/list_2_{i}.html" for i in range(1,73)]
urls1 = []
for url in tqdm(urls0):
    response = requests.get(url)
    if response.status_code == 200:    
        soup = BeautifulSoup(response.content, 'html.parser')
        html_doc = soup.get_text() 
        link_string = '\n'.join([str(link) for link in soup.find_all('a')])
        pattern = r"/cn/milesguo/[\w/]+\.html"
        matches = re.findall(pattern, link_string)
        urls2 = [f"https://gwins.org{x}" for x in matches]
        urls1 += urls2
        print(len(urls1))
    else:
        print(f"无法获取页面{url}，HTTP状态码：{response.status_code}")

In [ ]:
# 爬取所有2866个文章，并保存为文档
out_folder = "./txts"
if not os.path.isdir(out_folder): 
    os.mkdir("./txts")
for url in tqdm(urls1):
    pattern = r'\d+'
    id = re.search(pattern, url).group() # 获取网页编号
    response = requests.get(url)
    if response.status_code == 200:    
        soup = BeautifulSoup(response.content, 'html.parser')
    else:
        print(f"无法获取页面{url}，HTTP状态码：{response.status_code}")
        continue
    html_doc = soup.get_text()  # 获取网页中的纯文本内容
    html_doc = re.sub(r'\s+', ' ', html_doc) # 去掉多余空格
    
    file_path = os.path.join(out_folder, f"{id}.txt")
    with open(file_path, "w") as f:
        f.write(html_doc) # 保存

In [63]:
# 将长文档分解成1000字以内短文档. 因为openai sentence embedding ada 002 8000 input token, 2000汉字
def load_data_to_paragraphs(file):
    with open(file, "r") as f:
        data = f.read()
    pattern1 = r'^.*?内容梗概: '
    pattern2 = r' 友情链接：Gnews \| Gclubs \| Gfashion \| himalaya exchange \| gettr \| 法治基金 \| 新中国联邦辞典 \| $'
    data = re.sub(pattern1, "", data)
    data = re.sub(pattern2, "", data)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    txts = text_splitter.split_text(data)
    return txts

def sentence_embedding_batch(txts, id):
    """将list of text编码为sentence embedding 1538维"""
    l1 = []
    embs = get_embeddings(txts, engine="text-embedding-ada-002")
    for i, txt in enumerate(txts):
        label = f"{id}-{i}"
        emb = embs[i]
        l1.append((label, txt, emb))
    return l1

def encoding_file(in_file, output_dir):
    """将文档.txt文件编码为同名 embedding文件"""
    id = os.path.basename(in_file).split(".")[0]
    out_file = os.path.join(output_dir, id+".npz")
    txts = load_data_to_paragraphs(in_file)
    packs = sentence_embedding_batch(txts, id)
    serialized_data = pickle.dumps(packs)
    compressed_data = gzip.compress(serialized_data)
    with open(out_file, "wb") as file:
        file.write(compressed_data)

def encoding_files(input_dir = "./txts/", output_dir = "./emb"):
    """批量将文件夹下txt文件编码为同名 embedding文件"""
    files = [os.path.join(input_dir, x) for x in os.listdir(input_dir)]
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    for i, in_file in enumerate(tqdm(files)):
        encoding_file(in_file, output_dir)

def decoding_file(file):
    with open(file, "rb") as f:
        compressed_data = f.read()
    decompressed_data = gzip.decompress(compressed_data)
    l1 = pickle.loads(decompressed_data)
    return l1

def build_faiss_index(embs):
    import faiss
    d = 1536
    nlist = 100
    index = faiss.IndexFlatIP(d)
    #index = faiss.IndexIVFFlat(index, d, nlist)
    index.train(embs)
    index.add(embs)
    return index
    

def build_veactor_search_index(folder="./emb"):
    files = [os.path.join(folder, x) for x in os.listdir(folder)]
    dict1 = dict()
    i = 0
    embs = []
    for file in tqdm(files):
        l1 = decoding_file(file)
        for idx, txt, emb in l1:
            dict1[i] = {"idx": idx, "txt": txt, "emb": emb}
            embs.append(emb)
            i+=1
    embs = np.vstack(embs)
    embs /= np.linalg.norm(embs, ord=2, axis=-1, keepdims=True) + 1e-8
    faiss_index = build_faiss_index(embs)
    return embs, dict1, faiss_index

def text_search(query, faiss_index, dict1):
    if len(query)<10:
        query = f"这是一个关于{query}的句子"
    emb_query = get_embedding(query, engine="text-embedding-ada-002")
    emb_query = np.array(emb_query).reshape((1, -1))
    D, I = faiss_index.search(emb_query, k=3)
    print(D, I)
    txts = [(dict1[i]["idx"], dict1[i]["txt"]) for i in I[0]]
    return txts

def RAG_chatbot(txt, faiss_index, dict1):
    pass

In [40]:
#encoding_files(input_dir = "./txts/", output_dir = "./emb")
embs, dict1, faiss_index = build_veactor_search_index(folder="./emb")

100%|██████████████████████████████████████████████████████████████████████████████| 2866/2866 [00:04<00:00, 679.97it/s]


In [64]:
txt_query = "What happended to MH370 airplane? "
txts = text_search(txt_query, faiss_index, dict1)
txts

[[0.81078804 0.80657005 0.8030929 ]] [[18169 31816 12377]]


[('171-5',
  '更没有人关心他怎么去了法国去了南法，为什么不坐他的787而换了飞机？为什么临时到了酒店，临时又换酒店还换房间？为什么从过去的十几个人现在变成了七个人而不是六个人？为什么中间有一个人离开了团队？为什么现场一个照片一个视频没有？而且是内脏死，摔死没有任何的外伤，没有什么脑壳崩裂、外伤，没有人去谈这些细节。 更夸张的事情，在同一时间内，整个国内所有的媒体、放开的公布信息和所有的和王岐山、孟建柱有关的新闻突然大变样，中美贸易战都被放在了后面，就是“泼墨门”。 大家往前看，过去的几十年来围绕着中国一系列的具有重大意义的事件，总会发生让大家感觉到所有的海外所谓的大V们、公知们、民主民运分子们、有修养有文化的这些呼吁中国有法治中国、民主中国的这些精神领袖们、主席们、总统们、法官们，还有在天空中飘的白衣们（注：袁红兵又被称作袁白衣）都会围绕着那个核心事件。 这就是我告诉西方的所有的媒体的朋友，如果你们想要了解中国，你们要从过去的这29年中国的媒体运动关键时间点所发生的海内外媒体报道它的一致性和同质性和它的标准和它的重点和真相进行了解，那是了解中国最好的方式。包括我们在海外的几家媒体、几个主持人、几个老板，拿了黑钱瞪着眼睛说瞎话，瞪着眼睛在那块转移视线。 这就是这几天发生的海航事件、709律师事件，让我们所有人都应该学习和看到我们处在了一个什么样的环境，这个世界正在被黑暗所笼罩，几乎没有讲出真相的机会，一层又一层的特务网，一层又一层的“蓝金黄”，中国14亿人民已经被牢牢的被各种利益和盗国贼所绑架。 接下来大家会看到有关这些事情戏剧性的发展。这就像在两个月以前，五月份的时候我就说中美关系将出现重大事件，大家都看到了一系列的事件发生：中兴事件，中美贸易战的谈判，几次的反复前所未有。 但是大家看看有几个海外的媒体如实报道、认真报道、认真跟踪了？这里面的意义是如此之巨大，又有多少人去报道？但是这些人所有人讲的是什么？“泼墨门”、什么某某大V的热点事件，什么国内的某某什么丑闻，多少人去谈王岐山？又有多少人谈孟建柱？委内瑞拉、马来西亚有多大？又有多少人去关注？泰国的船翻是多少人死？为什么死？有多少人去关注？'),
 ('688-30',
  '我当时就跟他说你完全理解错了，我们说这个马航有什么事情是不正常的，从来没说是什么医生。大家去看一看马航为什么那些人在飞机上？不

Distances: [[0.99999994 0.9173957  0.90672827 0.9045879  0.9038532  0.897203
  0.89675575 0.8966185  0.89628506 0.89613855]]
Indices: [[    0     1  5983 21589 24700 29149 19581 18945 14504 27710]]


'0.28.1'